# YamboConvergence: automated G0W0 convergence

The highest-level workflow is represented by the ``YamboConvergence`` workchain, 
which implements the full automation of the convergence algorithm described in [Bonacci, M., Qiao, J., Spallanzani, N. et al. Towards high-throughput many-body perturbation theory: efficient algorithms and automated workflows. npj Comput Mater 9, 74 (2023)](https://doi.org/10.1038/s41524-023-01027-2). 
Simulations are organized on the fly, without any external user intervention. 
The purpose of this new proposed convergence algorithm is to obtain an accurate converged 
result doing the least possible number of calculations. This is possible if a reliable description of the convergence space is achieved, resulting also in a 
precise guess for the converged point, i.e. the converged parameters. The description of the space is performed by fitting some calculations that the workchain runs. 
A simple functional form of the space is assumed:

$f(\textbf{x}) = \prod_i^N \left( \frac{A_i}{x_i^{\alpha_i}} + b_i \right)$

In this way it is straightforward to compute first and second partial derivatives, and impose constraints on them to find the converged region of the space. 
The algorithm is specifically designed to solve the coupled convergence between 
summation over empty states (``BndsRnXp`` or ``BndsRnXs`` and ``GbndRnge`` for example) and PW expansion (``NGsBlkXp`` or ``NGsBlkXs``), but it can be used also to 
accelerate convergence tests with respect to the ``k-point mesh`` or ``FFTGvecs``, as we shall see later.

Each simulation is performed by calling the `YamboWorkflow` workchain, ensuring the automation at all levels.


### Example of workflow results:
The successful workflow will return the results of the convergence iterations, as well as a final converged calculation, from which we can parse the
converged parameters (they can be also found in the `infos` outputs of the workflow), and a complete story of all the calculations of the workflow with all the information provided.

To show how the convergence algorithm works, here we plot the convergences performed on 2D-hBN imposing a convergence threshold of 1% on the final gap. The convergence is 
performed with respect to ``NGsBlkXp`` (G_cut in the plot) and ``BndsRnXp`` = ``GbndRnge`` (Nb in the plot). 

.. figure:: ./images/2D_conv_hBN.png
    :width: 400px
    :align: center
    :height: 400px

We can observe that first simulations (black squares) are performed on a starting grid, the blue one. The algorithm decides then to perform another set of calculations on 
a shifted grid, as the fit perofmed to predict the space was not enough accurate. Next, a converged point is predicted, corresponding to the blue square, and it is explicitely computed. 
Using also the informations on that point, the algorithm understands that a new converged point can be the red one. This is then computed and verified to be the real converged one. In this 
way, convergence is efficiently achieved. 

All the calculations are automatically collected in a group, created using the structure formula, or can be collected in a specific pre-existing group if the input 
``builder.group_label`` is provided as Str datatype.

In [1]:
from aiida import orm, load_profile
load_profile()

from aiida.plugins import WorkflowFactory
from aiida.orm import QueryBuilder

from aiida_quantumespresso.common.types import ElectronicType

import yaml


qb = QueryBuilder()
qb.append(orm.Group, filters={'label': 'Silicon/bulk'}, tag='group')
qb.append(orm.StructureData, with_group='group')

loaded_structure_id = qb.all()[0][0].pk

# Read YAML file
with open("../configuration/codes_localhost.yaml", 'r') as stream:
    codes = yaml.safe_load(stream)
    
with open("../configuration/resources_localhost.yaml", 'r') as stream:
    resources = yaml.safe_load(stream)
    
options = {
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; if you want it, take ytheour previously nscf id (pk) to skip the DFT part.
    'structure_id':loaded_structure_id,
}
options.update(codes)

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

In [ ]:
YamboConvergence = WorkflowFactory('yambo.yambo.yamboconvergence')
builder = YamboConvergence.get_builder_from_protocol(
    pw_code = options['pwcode_id'],
    preprocessing_code = options['yamboprecode_id'],
    code = options['yambocode_id'],
    protocol=options['protocol'],
    protocol_qe=options['protocol'],
    structure=orm.load_node(options['structure_id']),
    overrides={},
    #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
    electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
    calc_type='gw', #or 'bse'; default is 'gw'
)

builder.ywfl.scf.pw.metadata.options = resources

builder.ywfl.nscf.pw.metadata.options = builder.ywfl.scf.pw.metadata.options
builder.ywfl.yres.yambo.metadata.options = builder.ywfl.scf.pw.metadata.options

In [10]:
#You can also try different protocols:
    
YamboConvergence.get_available_protocols()

{'fast': {'description': 'Fast protocol for a GW convergence: grid -> poor; thresholds -> poor'},
 'moderate': {'description': 'Moderate protocol for a GW convergence: grid -> enough good for standard materials; thresholds -> moderate (5 percent)'},
 'precise': {'description': 'precise protocol for a GW convergence: grid -> same as moderate; thresholds -> precise (1 percent)'},
 'molecule': {'description': 'Moderate protocol for a GW convergence in molecules'}}

### Overrides

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [14]:
overrides_scf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw':{
        'metadata':resources,
        },
    }

overrides_nscf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{},
                'ELECTRONS':{'diagonalization':'david'},
            },
             'metadata':resources,
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "NGsBlkXp": [4, "Ry"],
                    "FFTGvecs": [20, "Ry"],
                },
            },
        'metadata':resources,
        },
    
}


## providing additional convergence settings

In the following we provide additional convergence settings, namely:

- 'what': a list of quantities to be computed, following the same naming style of the `YamboWorkflow` additional parsing list;
- 'type': 'heavy', or 'cheap'; heavy keep the converged parameters as obtained in previous iterations; for example, if I convergence k-mesh and then bands, in the bands convergence we will use the converged k-mesh. This will make the calculation more and more computational demanding, but in the end we will obtain the true converged results, not only the converged parameters values.

In [16]:
overrides_wfl_settings = {
    'what':['gap_'],
    'type': 'heavy', #or cheap; heavy uses converged value for parameters that we are not converging in a given iteration.
    }

the quantities that we can converge are the ones that can be parsed by the 
``YamboWorkflow`` workchain: quasiparticle levels/gaps, and excitonic energies:

* 'gap_': gap as found from nscf calculation (may differ from the final GW band gap, in terms of value and k-point indexes);
* 'gap_GG': gap at the Gamma point;
* 'lowest_exciton': lowest exciton from BSE;
* 'brightest_exciton': brightest exciton from BSE.

In [17]:
#setting the overall overrides dictionary.

overrides = {
    'ywfl':{'scf':overrides_scf,'nscf':overrides_nscf,'yres':overrides_yambo},
    'workflow_settings':overrides_wfl_settings,
    }

In [18]:
builder = YamboConvergence.get_builder_from_protocol(
    pw_code = options['pwcode_id'],
    preprocessing_code = options['yamboprecode_id'],
    code = options['yambocode_id'],
    protocol=options['protocol'],
    protocol_qe=options['protocol'],
    structure= orm.load_node(options['structure_id']),
    overrides=overrides,
    pseudo_family = options["pseudo_family"]
    #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
    electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
    calc_type='gw', #or 'bse'; default is 'gw'
)



Summary of the main inputs:
BndsRnXp = 200
GbndRnge = 200
NGsBlkXp = 4 Ry
FFTGvecs = 20 Ry


kpoint mesh for nscf: [6, 6, 2]


/home/max/.conda/envs/aiida/lib/python3.9/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


### Inspecting the convergence space:

In [19]:
builder.parameters_space.get_list()

[{'var': ['FFTGvecs'],
  'start': 21,
  'stop': 58,
  'delta': 8,
  'max': 84,
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},
 {'var': ['kpoint_mesh'],
  'start': [4, 4, 2],
  'stop': [16, 16, 6],
  'delta': [3, 3, 3],
  'max': [30, 30, 10],
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},
 {'var': ['BndsRnXp', 'GbndRnge', 'NGsBlkXp'],
  'start': [80, 80, 2],
  'stop': [400, 400, 8],
  'delta': [50, 50, 1],
  'max': [600, 600, 10],
  'steps': 6,
  'max_iterations': 8,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_2D'}]

The workflow will follow such a list, from `FFTGvecs`, `kpoint_mesh` and then to `BndsRnXp, GbndRnge, NGsBlkXp`, performing each of the three iterations trying to convergence the corresponding parameters. Please note that the last step (`BndsRnXp, GbndRnge, NGsBlkXp`) will perform the convergence of the coupled parameters *BndsRnXp=GbndRnge* and *NGsBlkXp*. Depending on your system, a good convergence journey would be ['FFTGvecs'] -> ['BndsRnXp', 'GbndRnge', 'NGsBlkXp'] -> ['kpoint_mesh']. 

We can provide an ad hoc convergence space:

In [20]:
builder.parameters_space=orm.List(
    [{'var': ['FFTGvecs'],
      'start': 21,
      'stop': 58,
      'delta': 8,
      'max': 84,
      'steps': 4,
      'max_iterations': 4,
      'conv_thr': 10,
      'conv_thr_units': '%',
      'convergence_algorithm': 'new_algorithm_1D'},   #we are converging 1 parameter
     #{'var': ['kpoint_mesh'],
     # 'start': [4, 4, 2],
     # 'stop': [16, 16, 6],
     # 'delta': [3, 3, 3],
     # 'max': [30, 30, 10],
     # 'steps': 4,
     # 'max_iterations': 4,
     # 'conv_thr': 10,
     # 'conv_thr_units': '%',
     # 'convergence_algorithm': 'new_algorithm_1D'},
     {'var': ['BndsRnXp', 'GbndRnge', 'NGsBlkXp'],
      'start': [80, 80, 2],                           #starting values
      'stop': [400, 400, 8],                          #maximum values for the first grid
      'delta': [50, 50, 1],                           #grid spacing
      'max': [600, 600, 10],                          #maximum values for the largest grid possible
      'steps': 6,                                     #steps/calculation per iteration. For ['BndsRnXp', 'GbndRnge', 'NGsBlkXp'], always 6
      'max_iterations': 8,                            #maximum attempts
      'conv_thr': 10,                                 #converge threshold
      'conv_thr_units': '%',                          #converge threshold units: '%' is the relative error with respect to the most converged value; can be also 'eV'
      'convergence_algorithm': 'new_algorithm_2D'}],  #we are converging actually 2 parameters: bands
)

### Parameter-dependent resources

As you can imagine, increasing the parameters of the simulations may require also the change of the related computational resources, in order to be able to successfully perform them. 
Before the submission of the WorkChain, we can provide instructions on how to continously adapt the resources when parameters are changing.

In particular, we provide two dictionaries:
- parallelism instructions
- explicit resources instructions

In [21]:
dict_para_medium = {}
dict_para_medium['X_and_IO_CPU'] = '1 1 1 1 1'
dict_para_medium['X_and_IO_ROLEs'] = 'q k g c v'
dict_para_medium['DIP_CPU'] = '1 1 1'
dict_para_medium['DIP_ROLEs'] = 'k c v'
dict_para_medium['SE_CPU'] = '1 1 1'
dict_para_medium['SE_ROLEs'] = 'q qp b'

dict_res_medium = {
        "num_machines": 1,
        "num_mpiprocs_per_machine":1,
        "num_cores_per_mpiproc":1,
    }

dict_para_high = {}
dict_para_high['X_and_IO_CPU'] = '1 1 1 2 1' 
dict_para_high['X_and_IO_ROLEs'] = 'q k g c v'
dict_para_high['DIP_CPU'] = '1 2 1'
dict_para_high['DIP_ROLEs'] = 'k c v'
dict_para_high['SE_CPU'] = '1 1 2'
dict_para_high['SE_ROLEs'] = 'q qp b'

dict_res_high = {
        "num_machines": 1,
        "num_mpiprocs_per_machine":2,
        "num_cores_per_mpiproc":1,
    }

parallelism_instructions_manual = orm.Dict(dict={'manual' : {                                                            
                                                            'std_1':{
                                                                    'BndsRnXp':[1,100], #range for bands where to use the dict_para_medium and dict_res_medium instructions.
                                                                    'NGsBlkXp':[2,18],
                                                                    'parallelism':dict_para_medium,
                                                                    'resources':dict_res_medium,
                                                                    },
                                                            'std_2':{
                                                                    'BndsRnXp':[101,1000],
                                                                    'NGsBlkXp':[2,18],
                                                                    'parallelism':dict_para_high,
                                                                    'resources':dict_res_high,
                                                                    },}})

We can just provide, together with the resources, the `mode` which yambo will use to automatically set up its parallelism, if we are not sure on how to decide the explicit parallelism instructions.

In [22]:
parallelism_instructions_auto = orm.Dict(dict={'automatic' : {                                                            
                                                            'std_1':{
                                                                    'BndsRnXp':[1,100],
                                                                    'NGsBlkXp':[1,18],
                                                                    'mode':'balanced',
                                                                    'resources':dict_res_medium,
                                                                    },
                                                            'std_2':{
                                                                    'BndsRnXp':[101,1000],
                                                                    'NGsBlkXp':[1,18],
                                                                    'mode':'memory',
                                                                    'resources':dict_res_high,
                                                                    },}})

In [23]:
builder.parallelism_instructions = parallelism_instructions_auto

### Providing an AiiDA group where to collect all the convergence simulations

When `YamboConvergence` is submitted, it automatically creates a group where to put all the simulations. Each time a simulation is ready to be submitted, there is an internal check in the group, to understand
if an identical simulation has been already performed. In that case, the submission is skipped and we reuse the results to perform our analysis. This is a sort of ad-hoc [caching](https://aiida.readthedocs.io/projects/aiida-core/en/latest/topics/provenance/caching.html), which however does not duplicate the involved nodes. 

We prefer to just reuse the results as often even the retrieved files for yambo simulations are heavy (`ndb.QP` for example).

It is possible also to provide a custom group, by means of the `group_label` input String.

In [24]:
try:
    g = orm.load_group('tutorial/Silicon/convergence')
except:
    g = orm.Group('tutorial/Silicon/convergence')
    g.store()

In [25]:
builder.group_label = orm.Str('tutorial/Silicon/convergence') # verdi group create tutorial/Silicon/convergence; all calculationsc are added to the group

### Run

In [26]:
from aiida.engine import submit

In [27]:
run = None

In [28]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/home/max/.conda/envs/aiida/lib/python3.9/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: f5ec30d2-63e1-403e-85f1-b0708c231d15 (pk: 971) (aiida.workflows:yambo.yambo.yamboconvergence)


In [31]:
!verdi process report {run.pk}

2024-01-16 13:10:56 [334 | REPORT]: [971|YamboConvergence|start_workflow]: group: tutorial/hBN/convergence
2024-01-16 13:10:56 [335 | REPORT]: [971|YamboConvergence|start_workflow]: Workflow type: heavy; looking for convergence of ['gap_']
2024-01-16 13:10:56 [336 | REPORT]: [971|YamboConvergence|start_workflow]: Workflow initilization step completed, the parameters will be: ['FFTGvecs'].
2024-01-16 13:10:56 [337 | REPORT]: [971|YamboConvergence|has_to_continue]: Still iteration on ['FFTGvecs']
2024-01-16 13:10:56 [338 | REPORT]: [971|YamboConvergence|pre_needed]: {'FFTGvecs': [21, 37, 45, 58], 'BndsRnXp': [80, 400, 80, 180, 280, 400], 'NGsBlkXp': [2, 2, 8, 6, 4, 8], 'GbndRnge': [80, 400, 80, 180, 280, 400]}
2024-01-16 13:10:57 [339 | REPORT]: [971|YamboConvergence|pre_needed]: ['GW bands are: 200', 'found scf inputs from parent', 'found nscf inputs from parent\n']
2024-01-16 13:10:57 [340 | REPORT]: [971|YamboConvergence|next_step]: New parameters are: {'FFTGvecs': 21}
2024-01-16 13:1

# Output analysis.

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance: 

In [32]:
run = orm.load_node(971)

In [33]:
run.is_finished_ok

True

The converged parameters can be obtained via the "infos" output Dict:

In [34]:
run.outputs.infos.get_dict()

{'gap_': 6.2939792856842,
 'E_ref': 5.8912832924033,
 'BndsRnXp': 80,
 'FFTGvecs': 21,
 'GbndRnge': 80,
 'NGsBlkXp': 2.0}

The full convergence history can be visualized in a table form using pandas:

In [35]:
import pandas as pd

In [36]:
history = run.outputs.history.get_dict()

In [37]:
history_table = pd.DataFrame(history)

In [38]:
history_table

,gap_,uuid,failed,useful,BndsRnXp,FFTGvecs,GbndRnge,NGsBlkXp,global_step,parameters_studied
0,5.879072,1197c8e7-981d-4ccc-9108-eea06c3632d5,False,False,200,21,200,4,1,FFTGvecs
1,5.866152,aa433d73-ba7a-48f8-89ab-a95328ad5628,False,False,200,37,200,4,2,FFTGvecs
2,5.865376,6f05b410-90ed-4614-8296-cdef9cdf6a32,False,False,200,45,200,4,3,FFTGvecs
3,5.859457,2b34a618-2325-4845-80c8-f0c7c77beac5,False,False,200,58,200,4,4,FFTGvecs
4,6.293979,af272e4f-c5aa-47ef-93ec-c14ac7b0350c,False,True,80,21,80,2,5,"BndsRnXp, GbndRnge, NGsBlkXp"
5,6.280235,c7fd1b76-f95a-431b-8cbb-574ca935665d,False,False,400,21,400,2,6,"BndsRnXp, GbndRnge, NGsBlkXp"
6,5.946686,3987575d-c056-4e3e-8ff8-ba4f431a0312,False,False,80,21,80,8,7,"BndsRnXp, GbndRnge, NGsBlkXp"
7,5.896408,8be9bd35-6042-4b68-a32d-a83a78892a60,False,False,180,21,180,6,8,"BndsRnXp, GbndRnge, NGsBlkXp"
8,5.876000,74297842-de22-4847-aaee-2284eb77e471,False,False,280,21,280,4,9,"BndsRnXp, GbndRnge, NGsBlkXp"
9,5.925063,08e90f06-b702-4456-a10f-8dcb4ddfa11c,False,False,400,21,400,8,10,"BndsRnXp, GbndRnge, NGsBlkXp"


The converged calculations can be easily observed using:

In [39]:
history_table[history_table['useful']==True]

,gap_,uuid,failed,useful,BndsRnXp,FFTGvecs,GbndRnge,NGsBlkXp,global_step,parameters_studied
4,6.293979,af272e4f-c5aa-47ef-93ec-c14ac7b0350c,False,True,80,21,80,2,5,"BndsRnXp, GbndRnge, NGsBlkXp"


Result on the convergence path can be plotted using several plotting libraries, for examples here we are gonna use [plotly]((https://plotly.com/python/3d-line-plots/)) to observe the convergence between bands and plane wave cutoff for the screening matrix:

In [42]:
import plotly.express as px

b_G_history = history_table[history_table['parameters_studied']=="BndsRnXp, GbndRnge, NGsBlkXp"]

df = b_G_history
fig = px.scatter_3d(df, x="BndsRnXp", y="NGsBlkXp", z="gap_") #line=dict(color='red', width=2))
fig.show()